In [ ]:
import numpy as np
import cobra
import os
from os.path import join
from cobra.io import read_sbml_model
from cameo import fba

model = read_sbml_model('iYLI647.xml')
model.solver

modified = model.medium
modified["EX_glc_LPAREN_e_RPAREN_"]=20      # Glucose exchange 20 mmol/gDW/h
modified["EX_h2o_LPAREN_e_RPAREN_"]=1000    # H2O exchange
modified["EX_o2_LPAREN_e_RPAREN_"]=1000     # O2 exchange
modified["EX_h_LPAREN_e_RPAREN_"]=1000      # H exchange
modified["EX_inost_LPAREN_e_RPAREN_"]=1000     # myo Inositol exchange
modified["EX_k_LPAREN_e_RPAREN_"]=1000      # K exchange
modified["EX_na1_LPAREN_e_RPAREN_"]=1000    # Sodium exchange
modified["EX_nh4_LPAREN_e_RPAREN_"]=1000    # Ammonia exchange
modified["EX_pi_LPAREN_e_RPAREN_"]=1000     # Phosphate exchange
modified["EX_so4_LPAREN_e_RPAREN_"]=1000    # Sulfate exchange
modified["trehalose_c_tp"]=0                # trehalose c tp
model.medium = modified

model.genes.YALI0D17050g.knock_out()  # GGS1
model.genes.YALI0E05753g.knock_out()  # ERG20


from cobra import Metabolite

ggdp_c = Metabolite('ggdp_c', formula='C20H33O7P2', name='Geranylgeranyl diphosphate', compartment='c')

model.add_metabolites([
    Metabolite('ggdp_c', formula='C20H33O7P2', name='Geranylgeranyl diphosphate', compartment='c')   
    ])
model.add_boundary(model.metabolites.get_by_id('ggdp_c'), type="demand")  #reversible: exchange, sink / irreversible: demand

# ERG20, YALI0E05753g, Dimethylallyltranstransferase : dmpp_c + ipdp_c --> grdp_c + ppi_c
from cobra import Reaction

reaction1 = Reaction('DMATTmodi')
reaction1.name = 'Dimethylallyltranstransferase'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
reaction1.add_metabolites({
    model.metabolites.get_by_id('dmpp_c'): -1.0,
    model.metabolites.get_by_id('ipdp_c'): -1.0,
    model.metabolites.get_by_id('grdp_c'): 1.0,
    model.metabolites.get_by_id('ppi_c'): 1.0})
reaction1.gene_reaction_rule = 'YALI0E05753g'
model.add_reaction(reaction1)


# ERG20, YALI0E05753g, Geranyltranstransferase : grdp_c + ipdp_c --> frdp_c + ppi_c
reaction2 = Reaction('GRTTmodi')
reaction2.name = 'Geranyltranstransferase'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
reaction2.add_metabolites({
    model.metabolites.get_by_id('grdp_c'): -1.0,
    model.metabolites.get_by_id('ipdp_c'): -1.0,
    model.metabolites.get_by_id('frdp_c'): 1.0,
    model.metabolites.get_by_id('ppi_c'): 1.0})
reaction2.gene_reaction_rule = 'YALI0E05753g'
model.add_reaction(reaction2)


# GGS1, YALI0D17050g, Farnesyltransferase : frdp_c + ipdp_c --> ggdp_c + ppi_c
reaction3 = Reaction('GGS1_r0373')
reaction3.name = 'Farnesyltransferase'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({
    model.metabolites.get_by_id('frdp_c'): -1.0,
    model.metabolites.get_by_id('ipdp_c'): -1.0,
    model.metabolites.get_by_id('ggdp_c'): 1.0,
    model.metabolites.get_by_id('ppi_c'): 1.0})
reaction3.gene_reaction_rule = 'YALI0D17050g'
model.add_reaction(reaction3)

# add phytoene as metabolite in this model
phyto_c = Metabolite('phyto_c', formula = 'C40H64', name = 'phytoene-all_trans', compartment = 'c')
model.add_metabolites(phyto_c)
model.add_boundary(model.metabolites.get_by_id('phyto_c'), type="demand")

# carRP, phytoene synthase : ggdp_c + h_c --> phyto_c + ppi_c
reaction4 = Reaction('PHYT')
reaction4.name = 'phytoene_synthase'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({
    model.metabolites.get_by_id('ggdp_c'): -2.0,
    model.metabolites.get_by_id('h_c'): 2.0,
    model.metabolites.get_by_id('phyto_c'): 1.0,
    model.metabolites.get_by_id('ppi_c'): 2.0})
reaction4.gene_reaction_rule = 'McCarRP'
model.add_reaction(reaction4)

In [ ]:
#model.reactions.get_by_id('ALCD2x').knock_out() #etoh_c + nad_c <==> acald_c + h_c + nadh_c
#model.reactions.get_by_id('ALCD2m').knock_out() #etoh_m + nad_m <==> acald_m + h_m + nadh_m
#model.reactions.get_by_id('EX_fum_LPAREN_e_RPAREN_').knock_out() # fum_e -->
#model.reactions.get_by_id('EX_ala_L_LPAREN_e_RPAREN_').knock_out() # ala_L_e -->
#model.reactions.get_by_id('EX_val_L_LPAREN_e_RPAREN_').knock_out() # val_L_e -->
#model.reactions.get_by_id('EX_for_LPAREN_e_RPAREN_').knock_out() # for_e -->

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
import pandas as pd

df = pd.DataFrame()

data = (1, 2, 5, 10, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 500, 1000)

for i in data:
    modified["EX_glc_LPAREN_e_RPAREN_"] = i      # Glucose exchange 20 mmol/gDW/h
    modified["EX_o2_LPAREN_e_RPAREN_"] = 1000     # O2 exchange
    modified["EX_nh4_LPAREN_e_RPAREN_"]=1000    # Ammonia exchange
    
    model.medium = modified
    fseof = FSEOF(model)
    fs = fseof.run(target=model.reactions.PHYT, max_enforced_flux=0.9)
    fsdf = pd.DataFrame(fs.data_frame)

    #column 확인
    #fsdf.columns # column name 확인
    #fsdf.columns.map(type) # column name type 확인
    
    fsdf.columns = fsdf.columns.map(str)
    #'1' in fsdf.columns # True or False return: str=True, int=False 
    
    fsdf['flux_change' + str(i)] = fsdf['10'] - fsdf['1']
    fsdf1 = fsdf['flux_change' + str(i)]
    df = pd.merge(df, fsdf1, left_index=True, right_index=True, how='outer')
df    
df.to_csv ('iYLI647_FBA_FSEOF_PHYT_test4.csv', index=True, header=True)

In [ ]:
#modified["EX_glc_LPAREN_e_RPAREN_"] = 30      # Glucose exchange 20 mmol/gDW/h
#modified["EX_o2_LPAREN_e_RPAREN_"]=1000    # O2 exchange
#modified["EX_nh4_LPAREN_e_RPAREN_"]=1000
#model.medium = modified
#model.summary()

In [ ]:
#model.objective = 'biomass_C_limited'
#model.reactions.get_by_id('ALCD2x').knock_out() #etoh_c + nad_c <==> acald_c + h_c + nadh_c
#model.reactions.get_by_id('ALCD2m').knock_out() #etoh_m + nad_m <==> acald_m + h_m + nadh_m
#model.reactions.get_by_id('EX_fum_LPAREN_e_RPAREN_').knock_out() # fum_e -->
#model.reactions.get_by_id('EX_ala_L_LPAREN_e_RPAREN_').knock_out() # ala_L_e -->
#model.reactions.get_by_id('EX_val_L_LPAREN_e_RPAREN_').knock_out() # val_L_e -->
#model.reactions.get_by_id('EX_for_LPAREN_e_RPAREN_').knock_out() # for_e -->
#model.summary()

In [ ]:
#model.objective = 'PHYT'
#model.reactions.get_by_id('PHYT').upper_bound = 1000

In [ ]:
#biomass = model.reactions.biomass_C_limited
#phyt = model.reactions.PHYT

#new_objective = {biomass: 0.3, phyt: 0.7}
#model.optimize(new_objective=new_objective)
#model.summary()

In [ ]:
solution = model.optimize()
#solution.objective_value
#model.optimize().objective_value
#model.metabolites.atp_c.summary()

In [ ]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

model.objective = 'PHYT'
model.reactions.get_by_id('PHYT').upper_bound = 1000
linear_reaction_coefficients(model)

In [ ]:
model.optimize().objective_value

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
loop_reactions = [model.reactions.biomass_C_limited, model.reactions.PHYT]
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=False)

In [ ]:
model.summary()

NameError: name 'model' is not defined